# Mistral.rs Python API Cookbook


In [1]:
!git clone https://github.com/EricLBuehler/mistral.rs.git
!cd mistral.rs/mistralrs-pyo3
!pip install maturin[patchelf]
!maturin develop -r --features ...

Cloning into 'mistral.rs'...
remote: Enumerating objects: 19537, done.
remote: Counting objects: 100% (3797/3797), done.
remote: Compressing objects: 100% (1060/1060), done.
remote: Total 19537 (delta 2693), reused 3447 (delta 2560), pack-reused 15740
Receiving objects: 100% (19537/19537), 22.51 MiB | 8.42 MiB/s, done.
^Csolving deltas:  89% (12122/13620)
/bin/bash: line 1: cd: mistral.rs/mistralrs-pyo3: No such file or directory
💥 maturin failed
  Caused by: Can't find /home/ericbuehler/mistral.rs/examples/python/Cargo.toml (in /home/ericbuehler/mistral.rs/examples/python)


In [ ]:
from mistralrs import Runner, Which, ChatCompletionRequest, Message, Role

runner = Runner(
    which=Which.GGUF(
        tok_model_id="mistralai/Mistral-7B-Instruct-v0.1",
        quantized_model_id="TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
        quantized_filename="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
        tokenizer_json=None,
        repeat_last_n=64,
    )
)

res = runner.send_chat_completion_request(
    ChatCompletionRequest(
        model="mistral",
        messages=[Message(Role.User, "Tell me a story about the Rust type system.")],
        max_tokens=256,
        presence_penalty=1.0,
        top_p=0.1,
        temperature=0.1,
    )
)
print(res)

Lets walk through this code.
```python
from mistralrs import Runner, Which, ChatCompletionRequest, Message, Role
```

This imports the requires classes for our example. The `Runner` is a class which handles loading and running the model, which are enumerated by the `Which` class.

```python
runner = Runner(
    which=Which.GGUF(
        tok_model_id="mistralai/Mistral-7B-Instruct-v0.1",
        quantized_model_id="TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
        quantized_filename="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
        tokenizer_json=None,
        repeat_last_n=64,
    )
)
```

This tells the `Runner` to actually load the model. It will use a CUDA, Metal, or CPU device depending on what `features` you set during compilation: [here](https://github.com/EricLBuehler/mistral.rs?tab=readme-ov-file#supported-accelerators).

```python
res = runner.send_chat_completion_request(
    ChatCompletionRequest(
        model="mistral",
        messages=[Message(Role.User, "Tell me a story about the Rust type system.")],
        max_tokens=256,
        presence_penalty=1.0,
        top_p=0.1,
        temperature=0.1,
    )
)
print(res)
```

Now we actually send a request! We can specify the messages just like with an OpenAI API.

In [ ]:
from mistralrs import Runner, Which, ChatCompletionRequest, Message, Role, Architecture

runner = Runner(
    which=Which.Plain(
        model_id="mistralai/Mistral-7B-Instruct-v0.1",
        arch=Architecture.Mistral,
        tokenizer_json=None,
        repeat_last_n=64,
    )
)

res = runner.send_chat_completion_request(
    ChatCompletionRequest(
        model="mistral",
        messages=[Message(Role.User, "Tell me a story about the Rust type system.")],
        max_tokens=256,
        presence_penalty=1.0,
        top_p=0.1,
        temperature=0.1,
    )
)
print(res)

Lets walk through this code too and see the difference between loading a Plain model and loading a GGUF model.
```python
from mistralrs import Runner, Which, ChatCompletionRequest, Message, Role, Architecture
```

This imports the requires classes for our example. The `Runner` is a class which handles loading and running the model, which are enumerated by the `Which` class. Note that we also import the `Architecture` enum which controls the non-GGUF model's architecture.

```python
runner = Runner(
    which=Which.Plain(
        model_id="mistralai/Mistral-7B-Instruct-v0.1",
        arch=Architecture.Mistral,
        tokenizer_json=None,
        repeat_last_n=64,
    )
)
```

This tells the `Runner` to actually load the model as a Mistral architecture. It will use a CUDA, Metal, or CPU device depending on what `features` you set during compilation: [here](https://github.com/EricLBuehler/mistral.rs?tab=readme-ov-file#supported-accelerators).


## Loading a Mistral + GGUF model

In [ ]:
from mistralrs import Runner, Which

runner = Runner(
    which=Which.GGUF(
        tok_model_id="mistralai/Mistral-7B-Instruct-v0.1",
        quantized_model_id="TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
        quantized_filename="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
        tokenizer_json=None,
        repeat_last_n=64,
    )
)

## Loading a plain Mistral model

In [ ]:
from mistralrs import Runner, Which, Architecture

runner = Runner(
    which=Which.Plain(
        tok_model_id="mistralai/Mistral-7B-Instruct-v0.1",
        tokenizer_json=None,
        repeat_last_n=64,
        arch=Architecture.Mistral,
    )
)

## Loading an X-LoRA Zephyr model

In [ ]:
from mistralrs import Runner, Which

runner = Runner(
    which=Which.XLoraGGUF(
        tok_model_id=None, # Automatically determine from ordering file
        quantized_model_id="TheBloke/zephyr-7B-beta-GGUF",
        quantized_filename="zephyr-7b-beta.Q4_0.gguf",
        tokenizer_json=None,
        repeat_last_n=64,
        xlora_model_id="lamm-mit/x-lora",
        order="orderings/xlora-paper-ordering.json",
        tgt_non_granular_index=None,
    )
)


## Running the Runner

In [ ]:
from mistralrs import ChatCompletionRequest, Message, Role

res = runner.send_chat_completion_request(
    ChatCompletionRequest(
        model="mistral",
        messages=[Message(Role.User, "Tell me a story about the Rust type system.")],
        max_tokens=256,
        presence_penalty=1.0,
        top_p=0.1,
        temperature=0.1,
    )
)
print(res.choices[0].message.content)
print(res.usage)
